In [ ]:
# Application of FL task
from MLModel import *
from FLModel_row import *
from utils import *
from rdp_analysis import calibrating_sampled_gaussian
from privacy_analysis import calibrating_distp_noise

from torchvision import datasets, transforms
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

data_set = 'minst'

In [ ]:
n_clients = 4
n_classes = 10

In [ ]:
scattering, K, (h, w) = get_scatter_transform()
scattering.to(device)

def get_scattered_feature(dataset):
    scatters = []
    targets = []
    
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=256, shuffle=True, num_workers=1, pin_memory=True)

    
    for (data, target) in loader:
        data, target = data.to(device), target.to(device)
        if scattering is not None:
            data = scattering(data)
        scatters.append(data)
        targets.append(target)

    scatters = torch.cat(scatters, axis=0)
    targets = torch.cat(targets, axis=0)

    data = torch.utils.data.TensorDataset(scatters, targets)
    return data

def load_mnist():
    train = datasets.MNIST(root="~/data/", train=True, download=True, transform=transforms.ToTensor())
    test = datasets.MNIST(root="~/data/", train=False, download=True, transform=transforms.ToTensor())
    
    # get scattered features
    train = get_scattered_feature(train)
    test = get_scattered_feature(test)
    
    train_data = train[:][0].squeeze().cpu().float()
    train_label = train[:][1].cpu()
    
    test_data = test[:][0].squeeze().cpu().float()
    test_label = test[:][1].cpu()

    # Dirichlet distribution
    label_distribution = np.random.dirichlet([0.5]*n_clients, n_classes)
    class_idcs = [torch.where(train_label==y)[0].numpy() for y in range(n_classes)]
    client_idcs = [[] for _ in range(n_clients)]
    for c, fracs in zip(class_idcs, label_distribution):
        for i, idcs in enumerate(np.split(c, (np.cumsum(fracs)[:-1]*len(c)).astype(int))):
            client_idcs[i] += [idcs]

    client_idcs = [np.concatenate(idcs) for idcs in client_idcs]

    for i in range(4):
        print(client_idcs[i].shape)
    
    # split MNIST (training set) into non-iid data sets
    non_iid = []
    for i in range(n_clients):
        idx = client_idcs[i]
        d = train_data[idx]
        targets = train_label[idx].float()
        non_iid.append((d, targets))
    non_iid.append((test_data.float(), test_label.float()))
    return non_iid

In [ ]:
d = load_mnist()
torch.save(d, 'scattered-mnist-noniid.pt')

In [ ]:
ld = torch.load('scattered-mnist-noniid.pt')

In [ ]:
for i in range(5):
    print(torch.bincount(ld[i][1].int()))